In [49]:
import pickle
import numpy as np
import pandas as pd

In [50]:
# Cargar el modelo
with open('mejor_modelo_xgboost.pkl', 'rb') as f:
    model = pickle.load(f)
# Cargar los transformers
with open('transformer_scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

with open('transformer_target.pkl', 'rb') as f:
    target = pickle.load(f)

with open('transformer_one.pkl', 'rb') as f:
    one = pickle.load(f)

In [51]:
variables_one = ['MaritalStatus','JobRole','BusinessTravel', 'Department','EducationField']
data = {
    'Age': 51,
    'Attrition': 'Yes',
    'BusinessTravel': 'Travel_Rarely',
    'Department': 'Sales',
    'DistanceFromHome': 6,
    'Education': 2,
    'EducationField': 'Life Sciences',
    'Gender': 'Female',
    'JobLevel': 1,
    'JobRole': 'Healthcare Representative',
    'MaritalStatus': 'Married',
    'MonthlyIncome': 131160,
    'PercentSalaryHike': 11,
    'StockOptionLevel': 0,
    'TrainingTimesLastYear': 6,
    'YearsAtCompany': 1,
    'YearsSinceLastPromotion': 0,
    'YearsWithCurrManager': 0,
    'JobInvolvement': 3,
    'PerformanceRating': 3,
    'EnvironmentSatisfaction_knn': 3.0,  # Proxy of 'EnvironmentSatisfaction'
    'JobSatisfaction_knn': 4.0,  # Proxy of 'JobSatisfaction'
    'NumCompaniesWorked_knn': 1.0,  # Proxy of 'NumCompaniesWorked'
    'TotalWorkingYears_knn': 1.0  # Proxy of 'TotalWorkingYears'
}


In [52]:
df_pred = pd.DataFrame(data, index = [0])
df_pred.head(2)

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,Gender,JobLevel,JobRole,...,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,JobInvolvement,PerformanceRating,EnvironmentSatisfaction_knn,JobSatisfaction_knn,NumCompaniesWorked_knn,TotalWorkingYears_knn
0,51,Yes,Travel_Rarely,Sales,6,2,Life Sciences,Female,1,Healthcare Representative,...,6,1,0,0,3,3,3.0,4.0,1.0,1.0


In [53]:
col_numericas = df_pred.select_dtypes(include = np.number).columns
df_pred[col_numericas] = scaler.transform(df_pred[col_numericas])


In [54]:
# transformamos las categorias one-hot
df_one = pd.DataFrame(one.transform(df_pred[variables_one]).toarray(), columns=one.get_feature_names_out())
df_pred = pd.concat([df_pred, df_one], axis = 1)
df_pred.drop(columns = variables_one, axis = 1, inplace = True)

In [55]:
 # transformamos las categoricas de target
df_pred = target.transform(df_pred)

In [56]:
df_pred.drop(columns=['Attrition'],inplace=True)

In [57]:
prediction = model.predict(df_pred)
prob = model.predict_proba(df_pred)
print(prob)

[[0.95960397 0.04039601]]


In [58]:
prediction

array([0])